#Upset plots to compare the different methods of DEA, including the latest additions.

Mast, Limma, Deseq2 and FindAllMarkers seurat

In [0]:
#Load libraries
## Append the library folder
.libPaths(c("/dbfs/home/boriol@almirall.com/my_r_packages/bulkRNASeq_PBMCs_R4.3", .libPaths()))

library(DESeq2)
library(UpSetR)
library(tidyverse)
library(VennDiagram)
library(RColorBrewer)

.libPaths(c("/dbfs/home/jtrincado@almirall.com/my_r_packages/Seurat", .libPaths()))
library(openxlsx)
library(enrichplot)
library(ggplot2)

G_list <- readRDS("/dbfs/mnt/sandbox/RNASeq/PBMCs_IL4/pathways/G_list20240710.rds")
G_list_protein_coding <- G_list %>% filter(transcript_biotype == "protein_coding")
G_list_protein_coding <- G_list_protein_coding %>%unite("gene", ensembl_gene_id:hgnc_symbol, remove = FALSE)

#KC

##Reynolds

In [0]:
R.kc.de <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/Reynolds/LvsHC/reynolds_kc_LvsHC_allmarkers.xlsx")
rownames(R.kc.de) <- R.kc.de$gene


R.bulk.kc.de <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/Reynolds/LvsHC/pseudobulk/reynolds_kc_LvsHC_bulk_v2.xlsx")
rownames(R.bulk.kc.de) <- R.bulk.kc.de$gene


R.bulk.kc.limma <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/Reynolds/LvsHC/pseudobulk/reynolds_limma_results_kc.xlsx")
rownames(R.bulk.kc.limma) <- R.bulk.kc.limma$gene

R.kc.de.mast <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/Reynolds/LvsHC/MAST_method/reynolds_kc_LvsHC_allmarkers.xlsx")
rownames(R.kc.de.mast) <- R.kc.de.mast$gene

In [0]:
R.kc.de <- R.kc.de %>%
    mutate(avg_log2FC = ifelse(cluster == "healthy", -abs(avg_log2FC), avg_log2FC)) #It is done because the analysis was done only with positive results and to be able to differentiate healthy and lesional I assign that sign
R.kc.de.mast <- R.kc.de.mast %>%
    mutate(avg_log2FC = ifelse(cluster == "healthy", -abs(avg_log2FC), avg_log2FC)) #It is done because the analysis was done only with positive results and to be able to differentiate healthy and lesional I assign that sign

In [0]:
R.kc.de <- R.kc.de %>% inner_join(G_list_protein_coding, by = c("gene" = "hgnc_symbol"))
R.bulk.kc.de <- R.bulk.kc.de %>% inner_join(G_list_protein_coding, by = c("gene" = "hgnc_symbol"))
R.bulk.kc.limma <- R.bulk.kc.limma %>% inner_join(G_list_protein_coding, by = c("gene" = "hgnc_symbol"))
R.kc.de.mast <- R.kc.de.mast %>% inner_join(G_list_protein_coding, by = c("gene" = "hgnc_symbol"))

In [0]:
# DEGs

R.kc.de_filtered <- R.kc.de %>% filter(abs(avg_log2FC) > 1, p_val_adj < 0.05) %>% arrange(desc(abs(avg_log2FC)))
dim(R.kc.de_filtered)

R.bulk.kc.de_filtered <- R.bulk.kc.de %>% filter(abs(avg_log2FC) > 1, p_val_adj < 0.05) %>% arrange(desc(abs(avg_log2FC)))
dim(R.bulk.kc.de_filtered)

R.bulk.kc.limma_filtered <- R.bulk.kc.limma %>% filter(abs(logFC) > 1, adj.P.Val < 0.05) %>% arrange(desc(abs(logFC)))
dim(R.bulk.kc.limma_filtered)

R.kc.de.mast_filtered <- R.kc.de.mast %>% filter(abs(avg_log2FC) > 1, p_val_adj < 0.05) %>% arrange(desc(abs(avg_log2FC)))
dim(R.kc.de.mast_filtered)

## Reynolds KC Upset plot

In [0]:
# Plot upregulated
genes_kc <- list(
  "DEGs_KC_FindAllMarkers" = R.kc.de_filtered$gene, 
  "DEGs_KC_MAST" = R.kc.de.mast_filtered$gene, 
  "DEGs_KC_DESeq2" = R.bulk.kc.de_filtered$gene, 
  "DEGs_KC_Limma" = R.bulk.kc.limma_filtered$gene
)
genes_kc

In [0]:
options(repr.plot.width=1300, repr.plot.height=700)

In [0]:
upset(
  fromList(genes_kc), 
  order.by = "freq", 
  main.bar.color = "#393D47", 
  sets.bar.color = "darkgrey", 
  text.scale = c(1.5, 1.5, 1.2, 1.2, 1.5, 2))

##Alkon

In [0]:
A.kc.de <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/Alkon/alkon_kc_LvsHC_allmarkers.xlsx")
rownames(A.kc.de) <- A.kc.de$gene

A.bulk.kc.de <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/Alkon/pseudobulk/alkon_kc_LvsHC_bulk_v2.xlsx")
rownames(A.bulk.kc.de) <- A.bulk.kc.de$gene

A.bulk.kc.limma <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/Alkon/pseudobulk/alkon_limma_results_kc.xlsx")
rownames(A.bulk.kc.limma) <- A.bulk.kc.limma$gene

A.kc.de.mast <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/Alkon/MAST_method/alkon_kc_LvsHC_allmarkers.xlsx")
rownames(A.kc.de.mast) <- A.kc.de.mast$gene

In [0]:
A.kc.de <- A.kc.de %>%
    mutate(avg_log2FC = ifelse(cluster == "HC", -abs(avg_log2FC), avg_log2FC)) #It is done because the analysis was done only with positive results and to be able to differentiate healthy and lesional I assign that sign
A.kc.de.mast <- A.kc.de.mast %>%
    mutate(avg_log2FC = ifelse(cluster == "HC", -abs(avg_log2FC), avg_log2FC)) #It is done because the analysis was done only with positive results and to be able to differentiate healthy and lesional I assign that sign

In [0]:
A.kc.de <- A.kc.de %>% inner_join(G_list_protein_coding, by = c("gene" = "hgnc_symbol"))
A.bulk.kc.de <- A.bulk.kc.de %>% inner_join(G_list_protein_coding, by = c("gene" = "hgnc_symbol"))
A.bulk.kc.limma <- A.bulk.kc.limma %>% inner_join(G_list_protein_coding, by = c("gene" = "hgnc_symbol"))
A.kc.de.mast <- A.kc.de.mast %>% inner_join(G_list_protein_coding, by = c("gene" = "hgnc_symbol"))

In [0]:
# DEGs

A.kc.de_filtered <- A.kc.de %>% filter(abs(avg_log2FC) > 1, p_val_adj < 0.05) %>% arrange(desc(abs(avg_log2FC)))
dim(A.kc.de_filtered)

A.bulk.kc.de_filtered <- A.bulk.kc.de %>% filter(abs(avg_log2FC) > 1, p_val_adj < 0.05) %>% arrange(desc(abs(avg_log2FC)))
dim(A.bulk.kc.de_filtered)

A.bulk.kc.limma_filtered <- A.bulk.kc.limma %>% filter(abs(logFC) > 1, adj.P.Val < 0.05) %>% arrange(desc(abs(logFC)))
dim(A.bulk.kc.limma_filtered)

A.kc.de.mast_filtered <- A.kc.de.mast %>% filter(abs(avg_log2FC) > 1, p_val_adj < 0.05) %>% arrange(desc(abs(avg_log2FC)))
dim(A.kc.de.mast_filtered)

## Alkon KC Upset plot

In [0]:
# Plot upregulated
genes_kc <- list(
  "DEGs_KC_FindAllMarkers" = A.kc.de_filtered$gene, 
  "DEGs_KC_MAST" = A.kc.de.mast_filtered$gene, 
  "DEGs_KC_DESeq2" = A.bulk.kc.de_filtered$gene, 
  "DEGs_KC_Limma" = A.bulk.kc.limma_filtered$gene
)
genes_kc

In [0]:
options(repr.plot.width=1300, repr.plot.height=700)

In [0]:
upset(
  fromList(genes_kc), 
  order.by = "freq", 
  main.bar.color = "#393D47", 
  sets.bar.color = "darkgrey", 
  text.scale = c(1.5, 1.5, 1.2, 1.2, 1.5, 2))

In [0]:
options(repr.plot.width=1300, repr.plot.height=700)

In [0]:
upset(
  fromList(genes_kc), 
  order.by = "freq", 
  main.bar.color = "#393D47", 
  sets.bar.color = "darkgrey", 
  text.scale = c(1.5, 1.5, 1.2, 1.2, 1.5, 2))

##ALKON  VS REYNOLDS

In [0]:
genes_kc <- list(
  "Alkon_DEGs_KC_FindAllMarkers" = A.kc.de_filtered$gene, 
  "Alkon_DEGs_KC_MAST" = A.kc.de.mast_filtered$gene, 
  "Reynolds_DEGs_KC_FindAllMarkers" = R.kc.de_filtered$gene, 
  "Reynolds_DEGs_KC_MAST" = R.kc.de.mast_filtered$gene
)
genes_kc

options(repr.plot.width=1300, repr.plot.height=700)
upset(
  fromList(genes_kc), 
  order.by = "freq", 
  main.bar.color = "#393D47", 
  sets.bar.color = "darkgrey", 
  text.scale = c(1.5, 1.5, 1.2, 1.2, 1.5, 2))

In [0]:
genes_kc <- list(
  "Alkon_DEGs_KC_DESeq2" = A.bulk.kc.de_filtered$gene, 
  "Alkon_DEGs_KC_Limma" = A.bulk.kc.limma_filtered$gene,
  "Reynolds_DEGs_KC_DESeq2" = R.bulk.kc.de_filtered$gene, 
  "Reynolds_DEGs_KC_Limma" = R.bulk.kc.limma_filtered$gene
)
genes_kc

options(repr.plot.width=1300, repr.plot.height=700)
upset(
  fromList(genes_kc), 
  order.by = "freq", 
  main.bar.color = "#393D47", 
  sets.bar.color = "darkgrey", 
  text.scale = c(1.5, 1.5, 1.2, 1.2, 1.5, 2))

#Tcell

##Reynolds

In [0]:
##Reynolds
R.tcell.de <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/Reynolds/LvsHC/reynolds_LvsHC_tcell_allmarkers.xlsx")
rownames(R.tcell.de) <- R.tcell.de$gene

R.bulk.tcell.de <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/Reynolds/LvsHC/pseudobulk/reynolds_tcell_LvsHC_bulk_v2.xlsx")
rownames(R.bulk.tcell.de) <- R.bulk.tcell.de$gene

R.bulk.tcell.limma <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/Reynolds/LvsHC/pseudobulk/reynolds_limma_results_tcell.xlsx")
rownames(R.bulk.tcell.limma) <- R.bulk.tcell.limma$gene

R.tcell.de.mast <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/Reynolds/LvsHC/MAST_method/reynolds_LvsHC_tcell_allmarkers.xlsx")
rownames(R.tcell.de.mast) <- R.tcell.de.mast$gene
R.tcell.de <- R.tcell.de %>%
    mutate(avg_log2FC = ifelse(cluster == "healthy", -abs(avg_log2FC), avg_log2FC)) #It is done because the analysis was done only with positive results and to be able to differentiate healthy and lesional I assign that sign
R.tcell.de.mast <- R.tcell.de.mast %>%
    mutate(avg_log2FC = ifelse(cluster == "healthy", -abs(avg_log2FC), avg_log2FC)) #It is done because the analysis was done only with positive results and to be able to differentiate healthy and lesional I assign that sign
R.tcell.de <- R.tcell.de %>% inner_join(G_list_protein_coding, by = c("gene" = "hgnc_symbol"))
R.bulk.tcell.de <- R.bulk.tcell.de %>% inner_join(G_list_protein_coding, by = c("gene" = "hgnc_symbol"))
R.bulk.tcell.limma <- R.bulk.tcell.limma %>% inner_join(G_list_protein_coding, by = c("gene" = "hgnc_symbol"))
R.tcell.de.mast <- R.tcell.de.mast %>% inner_join(G_list_protein_coding, by = c("gene" = "hgnc_symbol"))
# DEGs

R.tcell.de_filtered <- R.tcell.de %>% filter(abs(avg_log2FC) > 1, p_val_adj < 0.05) %>% arrange(desc(abs(avg_log2FC)))
dim(R.tcell.de_filtered)

R.bulk.tcell.de_filtered <- R.bulk.tcell.de %>% filter(abs(avg_log2FC) > 1, p_val_adj < 0.05) %>% arrange(desc(abs(avg_log2FC)))
dim(R.bulk.tcell.de_filtered)

R.bulk.tcell.limma_filtered <- R.bulk.tcell.limma %>% filter(abs(logFC) > 1, adj.P.Val < 0.05) %>% arrange(desc(abs(logFC)))
dim(R.bulk.tcell.limma_filtered)

R.tcell.de.mast_filtered <- R.tcell.de.mast %>% filter(abs(avg_log2FC) > 1, p_val_adj < 0.05) %>% arrange(desc(abs(avg_log2FC)))
dim(R.tcell.de.mast_filtered)

## Reynolds TCell Upset plot
# Plot upregulated
genes_tcell <- list(
  "DEGs_TCell_FindAllMarkers" = R.tcell.de_filtered$gene, 
  "DEGs_TCell_MAST" = R.tcell.de.mast_filtered$gene, 
  "DEGs_TCell_DESeq2" = R.bulk.tcell.de_filtered$gene, 
  "DEGs_TCell_Limma" = R.bulk.tcell.limma_filtered$gene
)
genes_tcell
options(repr.plot.width=1300, repr.plot.height=700)
upset(
  fromList(genes_tcell), 
  order.by = "freq", 
  main.bar.color = "#393D47", 
  sets.bar.color = "darkgrey", 
  text.scale = c(1.5, 1.5, 1.2, 1.2, 1.5, 2))

##Alkon

In [0]:
##Alkon
A.tcell.de <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/Alkon/alkon_tcell_LvsHC_allmarkers.xlsx")
rownames(A.tcell.de) <- A.tcell.de$gene

A.bulk.tcell.de <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/Alkon/pseudobulk/alkon_tcell_LvsHC_bulk.xlsx")
rownames(A.bulk.tcell.de) <- A.bulk.tcell.de$gene

A.bulk.tcell.limma <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/Alkon/pseudobulk/alkon_limma_results_tcell.xlsx")
rownames(A.bulk.tcell.limma) <- A.bulk.tcell.limma$gene

A.tcell.de.mast <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/Alkon/MAST_method/alkon_LvsHC_tcell_allmarkers.xlsx")
rownames(A.tcell.de.mast) <- A.tcell.de.mast$gene
A.tcell.de <- A.tcell.de %>%
    mutate(avg_log2FC = ifelse(cluster == "HC", -abs(avg_log2FC), avg_log2FC)) #It is done because the analysis was done only with positive results and to be able to differentiate healthy and lesional I assign that sign
A.tcell.de.mast <- A.tcell.de.mast %>%
    mutate(avg_log2FC = ifelse(cluster == "HC", -abs(avg_log2FC), avg_log2FC)) #It is done because the analysis was done only with positive results and to be able to differentiate healthy and lesional I assign that sign
A.tcell.de <- A.tcell.de %>% inner_join(G_list_protein_coding, by = c("gene" = "hgnc_symbol"))
A.bulk.tcell.de <- A.bulk.tcell.de %>% inner_join(G_list_protein_coding, by = c("gene" = "hgnc_symbol"))
A.bulk.tcell.limma <- A.bulk.tcell.limma %>% inner_join(G_list_protein_coding, by = c("gene" = "hgnc_symbol"))
A.tcell.de.mast <- A.tcell.de.mast %>% inner_join(G_list_protein_coding, by = c("gene" = "hgnc_symbol"))
# DEGs

A.tcell.de_filtered <- A.tcell.de %>% filter(abs(avg_log2FC) > 1, p_val_adj < 0.05) %>% arrange(desc(abs(avg_log2FC)))
dim(A.tcell.de_filtered)

A.bulk.tcell.de_filtered <- A.bulk.tcell.de %>% filter(abs(avg_log2FC) > 1, p_val_adj < 0.05) %>% arrange(desc(abs(avg_log2FC)))
dim(A.bulk.tcell.de_filtered)

A.bulk.tcell.limma_filtered <- A.bulk.tcell.limma %>% filter(abs(logFC) > 1, adj.P.Val < 0.05) %>% arrange(desc(abs(logFC)))
dim(A.bulk.tcell.limma_filtered)

A.tcell.de.mast_filtered <- A.tcell.de.mast %>% filter(abs(avg_log2FC) > 1, p_val_adj < 0.05) %>% arrange(desc(abs(avg_log2FC)))
dim(A.tcell.de.mast_filtered)

## Alkon TCell Upset plot
genes_tcell <- list(
  "DEGs_TCell_FindAllMarkers" = A.tcell.de_filtered$gene, 
  "DEGs_TCell_MAST" = A.tcell.de.mast_filtered$gene, 
  "DEGs_TCell_DESeq2" = A.bulk.tcell.de_filtered$gene, 
  "DEGs_TCell_Limma" = A.bulk.tcell.limma_filtered$gene
)
genes_tcell

options(repr.plot.width=1300, repr.plot.height=700)
upset(
  fromList(genes_tcell), 
  order.by = "freq", 
  main.bar.color = "#393D47", 
  sets.bar.color = "darkgrey", 
  text.scale = c(1.5, 1.5, 1.2, 1.2, 1.5, 2))

##ALKON  VS REYNOLDS

In [0]:

genes_tcell <- list(
  "Alkon_DEGs_TCell_FindAllMarkers" = A.tcell.de_filtered$gene, 
  "Alkon_DEGs_TCell_MAST" = A.tcell.de.mast_filtered$gene, 
  "Reynolds_DEGs_TCell_FindAllMarkers" = R.tcell.de_filtered$gene, 
  "Reynolds_DEGs_TCell_MAST" = R.tcell.de.mast_filtered$gene
)
genes_tcell

options(repr.plot.width=1300, repr.plot.height=700)
upset(
  fromList(genes_tcell), 
  order.by = "freq", 
  main.bar.color = "#393D47", 
  sets.bar.color = "darkgrey", 
  text.scale = c(1.5, 1.5, 1.2, 1.2, 1.5, 2))

In [0]:

genes_tcell <- list(
  "Alkon_DEGs_TCell_DESeq2" = A.bulk.tcell.de_filtered$gene, 
  "Alkon_DEGs_TCell_Limma" = A.bulk.tcell.limma_filtered$gene,
  "Reynolds_DEGs_TCell_DESeq2" = R.bulk.tcell.de_filtered$gene, 
  "Reynolds_DEGs_TCell_Limma" = R.bulk.tcell.limma_filtered$gene
)
genes_tcell

options(repr.plot.width=1300, repr.plot.height=700)
upset(
  fromList(genes_tcell), 
  order.by = "freq", 
  main.bar.color = "#393D47", 
  sets.bar.color = "darkgrey", 
  text.scale = c(1.5, 1.5, 1.2, 1.2, 1.5, 2))